# Proyecto Final - Ciencia de datos en Python
#### Ingeniería de datos con Python
Jerson Ochoa - 15004111

### Scope del proyecto: 
Consiste en la elaboración de un pipeline de ingeniería de datos para la construcción de un DataWarehouse con base en el **análisis de la ejecución presupuestaria de los 340 municipios de Guatemala de los años 2019, 2020 y 2021**
Las fuentes de datos provienen del archivo de información del **Ministerio de Finanzas Guatemala** publicado en https://datos.minfin.gob.gt/. 
El objetivo del proyecto es analizar y comparar la ejecución presupuestaria (ingresos y egresos) de cada uno de los municipios  históricamente con los datos que provee el ministerio, la idea es preparar el pipeline para ingresar los datos conforme el ministerio los publica debido a que actualmente (año 2023) solo existen datos del 2021 para atrás. 
Los archivos publicados por el Ministerio de Finanzas se encuentran en formato CSV, se almacenarán en una base de datos RDS, se unificarán, se limpiarán y se transformarán los datos por medio de funciones de Python para formar un programa que realice el proceso, luego se creará una BD en formato estrella para completar el DataWarehouse. 

#### Distribución de las fuentes
- Se tienen 3 archivos csv, por lo tanto se cargará uno a una BD en RDS y los otros dos se cargarán a un bucket en S3 para cumplir con los requerimientos del proyecto

## Preguntas que se intentarán responder con el análisis de datos
1. ¿Cuál es el top 10 de municipios con el mayor ingreso global en 2021?
2. ¿Cuál es el top 10 de municipios con el menor ingreso global en 2021?
3. ¿Cuál es el top 10 de municipios con el mayor ingreso por tributos en 2021?
4. ¿Cuál es el top 10 de municipios con el mayor reajuste de presupuesto en 2021?
5. ¿Cuál es el top 10 de municipios con el mayor gasto en arrendamientos en 2021?

#### Nota: Las preguntas de análisis se hacen en base al año 2021, pero perfectamente deben poder ajustarse a cualquier año con datos cargados.

### Carga de librerías

In [31]:
import pandas as pd
import boto3
import psycopg2
import datetime
import numpy as np
import configparser
import mysql.connector as mysqlC
import io
import sys

### Carga de datos y exploración

In [2]:
# Datos 2019
datos_2019 = pd.read_csv('datasets/ingresos-municipales-2019.csv', sep=",", encoding='utf-8')
pd.set_option('display.max_columns', None)
datos_2019.head()

,Ejercicio,REGION,Código Entidad,Entidad,Código Departamento,Departamento,Código Clase,Clase,Código Sección,Sección,Código Grupo,Grupo,Código Recurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10100,IMPUESTOS DIRECTOS,10140,IMPUESTOS MUNICIPALES,10141,SOBRE LA TENENCIA DE PATRIMONIO,532810000.0,17877000.0,550687000.0,5.127175e+08,5.127175e+08
1,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10223,SOBRE DERIVADOS DEL PETRÓLEO,75000000.0,0.0,75000000.0,5.809833e+07,5.809833e+07
2,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10224,REGALÍAS,1200000.0,180000.0,1380000.0,1.001869e+06,1.001869e+06
3,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10282,SOBRE ESTABLECIMIENTOS DE SERVICIOS,57000.0,1000.0,58000.0,3.537000e+04,3.537000e+04
4,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,3078000.0,460000.0,3538000.0,2.956961e+06,2.956961e+06


In [3]:
# Datos 2020
datos_2020 = pd.read_csv('datasets/ingresos-municipales-2020.csv', sep=",", encoding='utf-8')
pd.set_option('display.max_columns', None)
datos_2020.head()

,Ejercicio,REGION,Código Entidad,Entidad,Código Departamento,Departamento,Código Clase,Clase,Código Sección,Sección,Código Grupo,Grupo,Código Recurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10100,IMPUESTOS DIRECTOS,10140,IMPUESTOS MUNICIPALES,10141,SOBRE LA TENENCIA DE PATRIMONIO,526223000.0,0.0,526223000.0,2.580033e+08,2.580033e+08
1,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10223,SOBRE DERIVADOS DEL PETRÓLEO,70000000.0,0.0,70000000.0,3.826994e+07,3.826994e+07
2,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10224,REGALÍAS,1770000.0,0.0,1770000.0,4.263171e+05,4.263171e+05
3,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10282,SOBRE ESTABLECIMIENTOS DE SERVICIOS,57000.0,0.0,57000.0,2.153500e+04,2.153500e+04
4,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,4230000.0,0.0,4230000.0,1.084353e+06,1.084353e+06


In [4]:
# Datos 2021
datos_2021 = pd.read_csv('datasets/ejecucion-presupuestaria-de-ingresos-dafim.csv', sep='\t', encoding='latin-1')
pd.set_option('display.max_columns', None)
datos_2021.head()

,Identificador,Ejercicio,Mes,Region,CodigoEntidad,Entidad,CodigoDepartamento,Departamento,CodigoClase,Clase,CodigoSeccion,Seccion,CodigoGrupo,Grupo,CodigoRecurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,1,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,16000,TRANSFERENCIAS CORRIENTES,16200,DEL SECTOR PÚBLICO,16210,DE LA ADMINISTRACIÓN CENTRAL,16210,DE LA ADMINISTRACIÓN CENTRAL,0.0,0.0,0.0,162969.26,162969.26
1,2,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10289,OTROS ARBITRIOS MUNICIPALES,0.0,0.0,0.0,3510.00,3510.00
2,3,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,0.0,0.0,0.0,180.00,180.00
3,4,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,11000,INGRESOS NO TRIBUTARIOS,11200,TASAS,11290,TASAS Y LICENCIAS VARIAS,11290,TASAS Y LICENCIAS VARIAS,0.0,0.0,0.0,29591.80,29591.80
4,5,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,11000,INGRESOS NO TRIBUTARIOS,11400,"ARRENDAMIENTO DE EDIFICIOS, EQUIPOS E INSTALAC...",11410,DE EDIFICIOS Y VIVIENDAS,11410,DE EDIFICIOS Y VIVIENDAS,0.0,0.0,0.0,180.00,180.00


### Cargar datos de 2021 a una BD en RDS

In [5]:
config = configparser.ConfigParser()
config.read('escec.cfg')

['escec.cfg']

#### Creamos instancia de RDS

In [6]:
aws_conn = boto3.client('rds', aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                    aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                    region_name='us-east-2')

#### Verificamos instancias de RDS disponibles

In [7]:
rdsInstanceIds = []

response = aws_conn.describe_db_instances()
for resp in response['DBInstances']:
    rdsInstanceIds.append(resp['DBInstanceIdentifier'])
    db_instance_status = resp['DBInstanceStatus']

print(f"DBInstanceIds {rdsInstanceIds}")

DBInstanceIds ['dw-db']


#### Creación de servicio RDS

In [8]:
rdsIdentifier = 'fuenteIngresosMuniMysql' 

In [9]:
try:
    response = aws_conn.create_db_instance(
            AllocatedStorage=10,
            DBName=config.get('RDS_MYSQL_F', 'DB_NAME'),
            DBInstanceIdentifier=rdsIdentifier,
            DBInstanceClass="db.t3.micro",
            Engine="mysql",
            MasterUsername=config.get('RDS_MYSQL_F', 'DB_USER'),
            MasterUserPassword=config.get('RDS_MYSQL_F', 'DB_PASSWORD'),
            Port=int(config.get('RDS_MYSQL_F', 'DB_PORT')),
            VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
            PubliclyAccessible=True
        )
    print(response)
except aws_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La Instancia de Base de Datos ya Existe.")

{'DBInstance': {'DBInstanceIdentifier': 'fuenteingresosmunimysql', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'mysql', 'DBInstanceStatus': 'creating', 'MasterUsername': 'root', 'DBName': 'fuente_ingresos_muni_mysql', 'AllocatedStorage': 10, 'PreferredBackupWindow': '09:38-10:08', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-08fdb378f31ed0d0f', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-09e2d36e7f45d1f52', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-0b304952bbcd0bc46', 'SubnetAvailabilityZone': {'Name': 'us-east-2b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-0b52320ca9b4e282e', 'SubnetAvailabilityZone': {'Name': 'us-east-2c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetI

#### Obtenemos url del host

In [10]:
try:
     instances = aws_conn.describe_db_instances(DBInstanceIdentifier=rdsIdentifier)
     RDS_MYSQL_F_HOST = instances.get('DBInstances')[0].get('Endpoint').get('Address')
     print(RDS_MYSQL_F_HOST)
except Exception as ex:
     print("La instancia de base de datos no existe o aun no se ha terminado de crear.")
     print(ex)

fuenteingresosmunimysql.c1dhtoyskhfb.us-east-2.rds.amazonaws.com


#### Conexión y creación de BD desde Python

In [15]:
import sql_query_fuente

try:
    myDw = mysqlC.connect(
    host=RDS_MYSQL_F_HOST, 
    user=config.get('RDS_MYSQL_F', 'DB_USER'),
    password=config.get('RDS_MYSQL_F', 'DB_PASSWORD'),
    database=config.get('RDS_MYSQL_F', 'DB_NAME')
    )

    mycursor = myDw.cursor()
    mycursor.execute(sql_query_fuente.DDL_QUERY, multi=True)
    myDw.commit()
    print("Data Warehouse Creado Exitosamente")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Data Warehouse Creado Exitosamente


#### Insertamos datos en la tabla Fuente de MYSQL

In [16]:
mysql_driver = f"""mysql+pymysql://{config.get('RDS_MYSQL_F', 'DB_USER')}:{config.get('RDS_MYSQL_F', 'DB_PASSWORD')}@{RDS_MYSQL_F_HOST}:{config.get('RDS_MYSQL_F', 'DB_PORT')}/{config.get('RDS_MYSQL_F', 'DB_NAME')}"""  

In [17]:
datos_2021.to_sql('ingresos_municipales', mysql_driver, index=False, if_exists='append')

59635

### Iniciamos lectura de fuentes

#### Lectura de BD MySQL en RDS

In [19]:
# Lectura de datos de 2021 en MySQL RDS
sql_query = 'SELECT * FROM ingresos_municipales'
df_fuente_2021 = pd.read_sql(sql_query, mysql_driver)
df_fuente_2021.head()

,Identificador,Ejercicio,Mes,Region,CodigoEntidad,Entidad,CodigoDepartamento,Departamento,CodigoClase,Clase,CodigoSeccion,Seccion,CodigoGrupo,Grupo,CodigoRecurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,1,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,16000,TRANSFERENCIAS CORRIENTES,16200,DEL SECTOR PÚBLICO,16210,DE LA ADMINISTRACIÓN CENTRAL,16210,DE LA ADMINISTRACIÓN CENTRAL,0.0,0.0,0.0,162969.26,162969.26
1,2,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10289,OTROS ARBITRIOS MUNICIPALES,0.0,0.0,0.0,3510.00,3510.00
2,3,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,0.0,0.0,0.0,180.00,180.00
3,4,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,11000,INGRESOS NO TRIBUTARIOS,11200,TASAS,11290,TASAS Y LICENCIAS VARIAS,11290,TASAS Y LICENCIAS VARIAS,0.0,0.0,0.0,29591.80,29591.80
4,5,2021,3,REGIÓN VII NOROCCIDENTE,12101407,MUNICIPALIDAD DE PATZITE,1400,QUICHE,11000,INGRESOS NO TRIBUTARIOS,11400,"ARRENDAMIENTO DE EDIFICIOS, EQUIPOS E INSTALAC...",11410,DE EDIFICIOS Y VIVIENDAS,11410,DE EDIFICIOS Y VIVIENDAS,0.0,0.0,0.0,180.00,180.00


#### Lectura de fuentes en S3

In [20]:
s3 = boto3.resource(
    service_name = 's3',
    region_name = 'us-east-2',
    aws_access_key_id = config.get('IAM', 'ACCESS_KEY'),
    aws_secret_access_key = config.get('IAM', 'SECRET_ACCESS_KEY')
)

In [21]:
for bucket in s3.buckets.all():
    S3_BUCKET_NAME = bucket.name
    print(bucket.name)

mybucketpapd


In [22]:
#extraemos todo lo que está en el bucket
remoteFileList = []
for objt in s3.Bucket(S3_BUCKET_NAME).objects.all():
    remoteFileList.append(objt.key)

remoteFileList

['divisas/',
 'divisas/conversion_divisas.xlsx',
 'ingresos_municipales/',
 'ingresos_municipales/ingresos-municipales-2019.csv',
 'ingresos_municipales/ingresos-municipales-2020.csv']

In [33]:
# Obtenemos el archivo del año 2019
try:
    file = s3.Bucket(S3_BUCKET_NAME).Object('ingresos_municipales/ingresos-municipales-2019.csv').get()
    data = file['Body'].read().decode('utf-8')
    df_fuente_2019 = pd.read_csv(io.StringIO(data))
except Exception as ex:
    print("No es un archivo.")
    print(ex)

df_fuente_2019.head()

,Ejercicio,REGION,Código Entidad,Entidad,Código Departamento,Departamento,Código Clase,Clase,Código Sección,Sección,Código Grupo,Grupo,Código Recurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10100,IMPUESTOS DIRECTOS,10140,IMPUESTOS MUNICIPALES,10141,SOBRE LA TENENCIA DE PATRIMONIO,532810000.0,17877000.0,550687000.0,5.127175e+08,5.127175e+08
1,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10223,SOBRE DERIVADOS DEL PETRÓLEO,75000000.0,0.0,75000000.0,5.809833e+07,5.809833e+07
2,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10224,REGALÍAS,1200000.0,180000.0,1380000.0,1.001869e+06,1.001869e+06
3,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10282,SOBRE ESTABLECIMIENTOS DE SERVICIOS,57000.0,1000.0,58000.0,3.537000e+04,3.537000e+04
4,2019,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,3078000.0,460000.0,3538000.0,2.956961e+06,2.956961e+06


In [34]:
# Obtenemos el archivo del año 2020
try:
    file = s3.Bucket(S3_BUCKET_NAME).Object('ingresos_municipales/ingresos-municipales-2020.csv').get()
    data = file['Body'].read().decode('utf-8')
    df_fuente_2020 = pd.read_csv(io.StringIO(data))
except Exception as ex:
    print("No es un archivo.")
    print(ex)

df_fuente_2020.head()

,Ejercicio,REGION,Código Entidad,Entidad,Código Departamento,Departamento,Código Clase,Clase,Código Sección,Sección,Código Grupo,Grupo,Código Recurso,Recurso,Asignado,Modificado,Vigente,Devengado,Percibido
0,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10100,IMPUESTOS DIRECTOS,10140,IMPUESTOS MUNICIPALES,10141,SOBRE LA TENENCIA DE PATRIMONIO,526223000.0,0.0,526223000.0,2.580033e+08,2.580033e+08
1,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10223,SOBRE DERIVADOS DEL PETRÓLEO,70000000.0,0.0,70000000.0,3.826994e+07,3.826994e+07
2,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10220,IMPUESTOS SOBRE PRODUCTOS INDUSTRIALES Y PRIMA...,10224,REGALÍAS,1770000.0,0.0,1770000.0,4.263171e+05,4.263171e+05
3,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10282,SOBRE ESTABLECIMIENTOS DE SERVICIOS,57000.0,0.0,57000.0,2.153500e+04,2.153500e+04
4,2020,REGIÓN I METROPOLITANA,12100101,MUNICIPALIDAD DE GUATEMALA,100,GUATEMALA,10000,INGRESOS TRIBUTARIOS,10200,IMPUESTOS INDIRECTOS,10280,ARBITRIOS MUNICIPALES,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS,4230000.0,0.0,4230000.0,1.084353e+06,1.084353e+06


### Generación de dimensiones

##### Dimensión Región

In [46]:
# Datos 2021
dimRegion = df_fuente_2021.loc[:, 'Region']
dimRegion.drop_duplicates(inplace=True)
dimRegion = pd.DataFrame(dimRegion)
dimRegion.reset_index(inplace=True)
dimRegion = dimRegion.drop(['index'], axis=1)
dimRegion

,Region
0,REGIÓN VII NOROCCIDENTE
1,REGIÓN III NORORIENTE
2,REGIÓN IV SURORIENTE
3,REGIÓN I METROPOLITANA
4,REGIÓN VI SUROCCIDENTE
5,REGIÓN V CENTRAL
6,REGIÓN II NORTE
7,REGIÓN VIII PETÉN


In [67]:
# Datos 2020
auxdf = df_fuente_2020.loc[:, 'REGION']
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'REGION':'Region'})
dimRegion.merge(auxdf, how='inner', on='Region')
dimRegion

,Region
0,REGIÓN VII NOROCCIDENTE
1,REGIÓN III NORORIENTE
2,REGIÓN IV SURORIENTE
3,REGIÓN I METROPOLITANA
4,REGIÓN VI SUROCCIDENTE
5,REGIÓN V CENTRAL
6,REGIÓN II NORTE
7,REGIÓN VIII PETÉN


In [68]:
# Datos 2019
auxdf = df_fuente_2019.loc[:, 'REGION']
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'REGION':'Region'})
dimRegion.merge(auxdf, how='inner', on='Region')
dimRegion

,Region
0,REGIÓN VII NOROCCIDENTE
1,REGIÓN III NORORIENTE
2,REGIÓN IV SURORIENTE
3,REGIÓN I METROPOLITANA
4,REGIÓN VI SUROCCIDENTE
5,REGIÓN V CENTRAL
6,REGIÓN II NORTE
7,REGIÓN VIII PETÉN


##### Dimensión Entidad

In [79]:
# Datos 2021
dimEntidad = df_fuente_2021.loc[:, ['CodigoEntidad', 'Entidad']]
dimEntidad.drop_duplicates(inplace=True)
dimEntidad = pd.DataFrame(dimEntidad)
dimEntidad.reset_index(inplace=True)
dimEntidad = dimEntidad.drop(['index'], axis=1)
dimEntidad

,CodigoEntidad,Entidad
0,12101407,MUNICIPALIDAD DE PATZITE
1,12101904,MUNICIPALIDAD DE GUALAN
2,12100606,MUNICIPALIDAD DE ORATORIO
3,12101906,MUNICIPALIDAD DE USUMATLAN
4,12100607,MUNICIPALIDAD DE SAN JUAN TECUACO
...,...,...
335,12101402,MUNICIPALIDAD DE CHICHE
336,12101404,MUNICIPALIDAD DE ZACUALPA
337,12101405,MUNICIPALIDAD DE CHAJUL
338,12101406,MUNICIPALIDAD DE CHICHICASTENANGO


In [85]:
# Datos 2020
auxdf = df_fuente_2020.loc[:, ['Código Entidad', 'Entidad']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Entidad':'CodigoEntidad'})
dimEntidad.merge(auxdf, how='inner', on='CodigoEntidad')
dimEntidad

,CodigoEntidad,Entidad
0,12101407,MUNICIPALIDAD DE PATZITE
1,12101904,MUNICIPALIDAD DE GUALAN
2,12100606,MUNICIPALIDAD DE ORATORIO
3,12101906,MUNICIPALIDAD DE USUMATLAN
4,12100607,MUNICIPALIDAD DE SAN JUAN TECUACO
...,...,...
335,12101402,MUNICIPALIDAD DE CHICHE
336,12101404,MUNICIPALIDAD DE ZACUALPA
337,12101405,MUNICIPALIDAD DE CHAJUL
338,12101406,MUNICIPALIDAD DE CHICHICASTENANGO


In [94]:
# Datos 2019
auxdf = df_fuente_2019.loc[:, ['Código Entidad', 'Entidad']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Entidad':'CodigoEntidad'})
dimEntidad.merge(auxdf, how='inner', on='CodigoEntidad')
dimEntidad

,CodigoEntidad,Entidad
0,12101407,MUNICIPALIDAD DE PATZITE
1,12101904,MUNICIPALIDAD DE GUALAN
2,12100606,MUNICIPALIDAD DE ORATORIO
3,12101906,MUNICIPALIDAD DE USUMATLAN
4,12100607,MUNICIPALIDAD DE SAN JUAN TECUACO
...,...,...
335,12101402,MUNICIPALIDAD DE CHICHE
336,12101404,MUNICIPALIDAD DE ZACUALPA
337,12101405,MUNICIPALIDAD DE CHAJUL
338,12101406,MUNICIPALIDAD DE CHICHICASTENANGO


##### Dimensión Departamento

In [88]:
# Datos 2021
dimDepartamento = df_fuente_2021.loc[:, ['CodigoDepartamento', 'Departamento']]
dimDepartamento.drop_duplicates(inplace=True)
dimDepartamento = pd.DataFrame(dimDepartamento)
dimDepartamento.reset_index(inplace=True)
dimDepartamento = dimDepartamento.drop(['index'], axis=1)
dimDepartamento

,CodigoDepartamento,Departamento
0,1400,QUICHE
1,1900,ZACAPA
2,600,SANTA ROSA
3,100,GUATEMALA
4,1800,IZABAL
5,700,SOLOLÁ
6,1200,SAN MARCOS
7,400,CHIMALTENANGO
8,300,SACATEPÉQUEZ
9,1300,HUEHUETENANGO


In [93]:
# Datos 2020
auxdf = df_fuente_2020.loc[:, ['Código Departamento', 'Departamento']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Departamento':'CodigoDepartamento'})
dimDepartamento.merge(auxdf, how='inner', on='CodigoDepartamento')
dimDepartamento

,CodigoDepartamento,Departamento
0,1400,QUICHE
1,1900,ZACAPA
2,600,SANTA ROSA
3,100,GUATEMALA
4,1800,IZABAL
5,700,SOLOLÁ
6,1200,SAN MARCOS
7,400,CHIMALTENANGO
8,300,SACATEPÉQUEZ
9,1300,HUEHUETENANGO


In [95]:
# Datos 2019
auxdf = df_fuente_2019.loc[:, ['Código Departamento', 'Departamento']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Departamento':'CodigoDepartamento'})
dimDepartamento.merge(auxdf, how='inner', on='CodigoDepartamento')
dimDepartamento

,CodigoDepartamento,Departamento
0,1400,QUICHE
1,1900,ZACAPA
2,600,SANTA ROSA
3,100,GUATEMALA
4,1800,IZABAL
5,700,SOLOLÁ
6,1200,SAN MARCOS
7,400,CHIMALTENANGO
8,300,SACATEPÉQUEZ
9,1300,HUEHUETENANGO


##### Dimension Clase

In [99]:
# Datos 2021
dimClase = df_fuente_2021.loc[:, ['CodigoClase', 'Clase']]
dimClase.drop_duplicates(inplace=True)
dimClase = pd.DataFrame(dimClase)
dimClase.reset_index(inplace=True)
dimClase = dimClase.drop(['index'], axis=1)
dimClase

,CodigoClase,Clase
0,16000,TRANSFERENCIAS CORRIENTES
1,10000,INGRESOS TRIBUTARIOS
2,11000,INGRESOS NO TRIBUTARIOS
3,14000,INGRESOS DE OPERACION
4,13000,VTA. DE BIENES Y SERV. DE LA ADM. PÚBLICA
5,17000,TRANSFERENCIAS DE CAPITAL
6,15000,RENTAS DE LA PROPIEDAD
7,23000,DISMINUCIÓN DE OTROS ACTIVOS FINANCIEROS
8,24000,ENDEUDAMIENTO PÚBLICO INTERNO
9,18000,RECURSOS PROPIOS DE CAPITAL


In [102]:
# Datos 2020
auxdf = df_fuente_2020.loc[:, ['Código Clase', 'Clase']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Clase':'CodigoClase'})
dimClase.merge(auxdf, how='inner', on='CodigoClase')
dimClase

,CodigoClase,Clase
0,16000,TRANSFERENCIAS CORRIENTES
1,10000,INGRESOS TRIBUTARIOS
2,11000,INGRESOS NO TRIBUTARIOS
3,14000,INGRESOS DE OPERACION
4,13000,VTA. DE BIENES Y SERV. DE LA ADM. PÚBLICA
5,17000,TRANSFERENCIAS DE CAPITAL
6,15000,RENTAS DE LA PROPIEDAD
7,23000,DISMINUCIÓN DE OTROS ACTIVOS FINANCIEROS
8,24000,ENDEUDAMIENTO PÚBLICO INTERNO
9,18000,RECURSOS PROPIOS DE CAPITAL


In [103]:
# Datos 2019
auxdf = df_fuente_2019.loc[:, ['Código Clase', 'Clase']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Clase':'CodigoClase'})
dimClase.merge(auxdf, how='inner', on='CodigoClase')
dimClase

,CodigoClase,Clase
0,16000,TRANSFERENCIAS CORRIENTES
1,10000,INGRESOS TRIBUTARIOS
2,11000,INGRESOS NO TRIBUTARIOS
3,14000,INGRESOS DE OPERACION
4,13000,VTA. DE BIENES Y SERV. DE LA ADM. PÚBLICA
5,17000,TRANSFERENCIAS DE CAPITAL
6,15000,RENTAS DE LA PROPIEDAD
7,23000,DISMINUCIÓN DE OTROS ACTIVOS FINANCIEROS
8,24000,ENDEUDAMIENTO PÚBLICO INTERNO
9,18000,RECURSOS PROPIOS DE CAPITAL


##### Dimensión Sección

In [104]:
# Datos 2021
dimSeccion = df_fuente_2021.loc[:, ['CodigoSeccion', 'Seccion']]
dimSeccion.drop_duplicates(inplace=True)
dimSeccion = pd.DataFrame(dimSeccion)
dimSeccion.reset_index(inplace=True)
dimSeccion = dimSeccion.drop(['index'], axis=1)
dimSeccion

,CodigoSeccion,Seccion
0,16200,DEL SECTOR PÚBLICO
1,10200,IMPUESTOS INDIRECTOS
2,11200,TASAS
3,11400,"ARRENDAMIENTO DE EDIFICIOS, EQUIPOS E INSTALAC..."
4,11900,OTROS INGRESOS NO TRIBUTARIOS
5,14200,VENTA DE SERVICIOS
6,13200,VENTA DE SERVICIOS
7,17200,DEL SECTOR PÚBLICO
8,15100,INTERESES
9,10100,IMPUESTOS DIRECTOS


In [105]:
# Datos 2020
auxdf = df_fuente_2020.loc[:, ['Código Sección', 'Sección']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Sección':'CodigoSeccion'})
auxdf = auxdf.rename(columns={'Sección':'Seccion'})
dimSeccion.merge(auxdf, how='inner', on='CodigoSeccion')
dimSeccion

,CodigoSeccion,Seccion
0,16200,DEL SECTOR PÚBLICO
1,10200,IMPUESTOS INDIRECTOS
2,11200,TASAS
3,11400,"ARRENDAMIENTO DE EDIFICIOS, EQUIPOS E INSTALAC..."
4,11900,OTROS INGRESOS NO TRIBUTARIOS
5,14200,VENTA DE SERVICIOS
6,13200,VENTA DE SERVICIOS
7,17200,DEL SECTOR PÚBLICO
8,15100,INTERESES
9,10100,IMPUESTOS DIRECTOS


In [106]:
# Datos 2019
auxdf = df_fuente_2019.loc[:, ['Código Sección', 'Sección']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Sección':'CodigoSeccion'})
auxdf = auxdf.rename(columns={'Sección':'Seccion'})
dimSeccion.merge(auxdf, how='inner', on='CodigoSeccion')
dimSeccion

,CodigoSeccion,Seccion
0,16200,DEL SECTOR PÚBLICO
1,10200,IMPUESTOS INDIRECTOS
2,11200,TASAS
3,11400,"ARRENDAMIENTO DE EDIFICIOS, EQUIPOS E INSTALAC..."
4,11900,OTROS INGRESOS NO TRIBUTARIOS
5,14200,VENTA DE SERVICIOS
6,13200,VENTA DE SERVICIOS
7,17200,DEL SECTOR PÚBLICO
8,15100,INTERESES
9,10100,IMPUESTOS DIRECTOS


##### Dimensión Grupo

In [107]:
# Datos 2021
dimGrupo = df_fuente_2021.loc[:, ['CodigoGrupo', 'Grupo']]
dimGrupo.drop_duplicates(inplace=True)
dimGrupo = pd.DataFrame(dimGrupo)
dimGrupo.reset_index(inplace=True)
dimGrupo = dimGrupo.drop(['index'], axis=1)
dimGrupo

,CodigoGrupo,Grupo
0,16210,DE LA ADMINISTRACIÓN CENTRAL
1,10280,ARBITRIOS MUNICIPALES
2,11290,TASAS Y LICENCIAS VARIAS
3,11410,DE EDIFICIOS Y VIVIENDAS
4,11990,OTROS INGRESOS NO TRIBUTARIOS
5,14240,SERVICIOS PÚBLICOS MUNICIPALES
6,13260,SERVICIOS ADMINISTRATIVOS MUNICIPALES
7,17210,DE LA ADMINISTRACIÓN CENTRAL
8,17220,DE ENTIDADES DESCENTRALIZADAS Y AUTÓNOMAS NO F...
9,15130,POR DEPOSITOS


In [108]:
# Datos 2020
auxdf = df_fuente_2020.loc[:, ['Código Grupo', 'Grupo']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Grupo':'CodigoGrupo'})
auxdf = auxdf.rename(columns={'Grupo':'Grupo'})
dimGrupo.merge(auxdf, how='inner', on='CodigoGrupo')
dimGrupo

,CodigoGrupo,Grupo
0,16210,DE LA ADMINISTRACIÓN CENTRAL
1,10280,ARBITRIOS MUNICIPALES
2,11290,TASAS Y LICENCIAS VARIAS
3,11410,DE EDIFICIOS Y VIVIENDAS
4,11990,OTROS INGRESOS NO TRIBUTARIOS
5,14240,SERVICIOS PÚBLICOS MUNICIPALES
6,13260,SERVICIOS ADMINISTRATIVOS MUNICIPALES
7,17210,DE LA ADMINISTRACIÓN CENTRAL
8,17220,DE ENTIDADES DESCENTRALIZADAS Y AUTÓNOMAS NO F...
9,15130,POR DEPOSITOS


In [109]:
# Datos 2019
auxdf = df_fuente_2019.loc[:, ['Código Grupo', 'Grupo']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Grupo':'CodigoGrupo'})
auxdf = auxdf.rename(columns={'Grupo':'Grupo'})
dimGrupo.merge(auxdf, how='inner', on='CodigoGrupo')
dimGrupo

,CodigoGrupo,Grupo
0,16210,DE LA ADMINISTRACIÓN CENTRAL
1,10280,ARBITRIOS MUNICIPALES
2,11290,TASAS Y LICENCIAS VARIAS
3,11410,DE EDIFICIOS Y VIVIENDAS
4,11990,OTROS INGRESOS NO TRIBUTARIOS
5,14240,SERVICIOS PÚBLICOS MUNICIPALES
6,13260,SERVICIOS ADMINISTRATIVOS MUNICIPALES
7,17210,DE LA ADMINISTRACIÓN CENTRAL
8,17220,DE ENTIDADES DESCENTRALIZADAS Y AUTÓNOMAS NO F...
9,15130,POR DEPOSITOS


##### Dimensión Recurso

In [110]:
# Datos 2021
dimRecurso = df_fuente_2021.loc[:, ['CodigoRecurso', 'Recurso']]
dimRecurso.drop_duplicates(inplace=True)
dimRecurso = pd.DataFrame(dimRecurso)
dimRecurso.reset_index(inplace=True)
dimRecurso = dimRecurso.drop(['index'], axis=1)
dimRecurso

,CodigoRecurso,Recurso
0,16210,DE LA ADMINISTRACIÓN CENTRAL
1,10289,OTROS ARBITRIOS MUNICIPALES
2,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS
3,11290,TASAS Y LICENCIAS VARIAS
4,11410,DE EDIFICIOS Y VIVIENDAS
...,...,...
68,11720,ORIGINADOS POR INGRESOS DE PREVISIÓN SOCIAL
69,10225,HIDROCARBUROS COMPARTIBLES
70,13111,VENTA DE INSUMOS AGRÍCOLAS
71,15212,DE EMPRESAS PÚBLICAS MUNICIPALES


In [111]:
# Datos 2020
auxdf = df_fuente_2020.loc[:, ['Código Recurso', 'Recurso']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Recurso':'CodigoRecurso'})
auxdf = auxdf.rename(columns={'Recurso':'Recurso'})
dimRecurso.merge(auxdf, how='inner', on='CodigoRecurso')
dimRecurso

,CodigoRecurso,Recurso
0,16210,DE LA ADMINISTRACIÓN CENTRAL
1,10289,OTROS ARBITRIOS MUNICIPALES
2,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS
3,11290,TASAS Y LICENCIAS VARIAS
4,11410,DE EDIFICIOS Y VIVIENDAS
...,...,...
68,11720,ORIGINADOS POR INGRESOS DE PREVISIÓN SOCIAL
69,10225,HIDROCARBUROS COMPARTIBLES
70,13111,VENTA DE INSUMOS AGRÍCOLAS
71,15212,DE EMPRESAS PÚBLICAS MUNICIPALES


In [112]:
# Datos 2019
auxdf = df_fuente_2019.loc[:, ['Código Recurso', 'Recurso']]
auxdf.drop_duplicates(inplace=True)
auxdf = pd.DataFrame(auxdf)
auxdf = auxdf.rename(columns={'Código Recurso':'CodigoRecurso'})
auxdf = auxdf.rename(columns={'Recurso':'Recurso'})
dimRecurso.merge(auxdf, how='inner', on='CodigoRecurso')
dimRecurso

,CodigoRecurso,Recurso
0,16210,DE LA ADMINISTRACIÓN CENTRAL
1,10289,OTROS ARBITRIOS MUNICIPALES
2,10284,SOBRE DIVERSIONES Y ESPECTÁCULOS
3,11290,TASAS Y LICENCIAS VARIAS
4,11410,DE EDIFICIOS Y VIVIENDAS
...,...,...
68,11720,ORIGINADOS POR INGRESOS DE PREVISIÓN SOCIAL
69,10225,HIDROCARBUROS COMPARTIBLES
70,13111,VENTA DE INSUMOS AGRÍCOLAS
71,15212,DE EMPRESAS PÚBLICAS MUNICIPALES
